<!-- dom:TITLE: Introduction to Python (MOD510): Functions -->
# Introduction to Python (MOD510): Functions
<!-- dom:AUTHOR: Oddbjørn Nødland -->
<!-- Author: -->  
**Oddbjørn Nødland**

Date: **Aug 20, 2019**

In [1]:
%matplotlib inline

import numpy as np
import math
import matplotlib.pyplot as plt

**Summary.** The aim of this workbook is to provide a rapid introduction to Python
function objects, and to show some examples of how you can work with
them.








## Functions

Functions provide an incredibly useful way to split a large program into
smaller, more manageable pieces, and they serve to reduce unnecessary
code duplication. They can communicate with the 'outside' by taking any number
of inputs and/or returning any number of outputs, or else they can simply
execute some code local to the function itself.

Python comes with many built-in functions, and many more are provided by
numerical packages such as NumPy and SciPy. It is also easy to define
your own. Examples of how to define and call functions are demonstrated
below:

In [2]:
# Examples of working with functions in Python:

# Functions of one variable:
def celsius_to_fahrenheit(temperature_celsius):
    return (9.0/5)*temperature_celsius + 32.0

def fahrenheit_to_celsius(temperature_fahrenheit):
    return (5.0/9)*(temperature_fahrenheit-32.0)

print('Freezing point of water: 0 C = {} F.'.format(celsius_to_fahrenheit(0.0)))
print('Boiling point of water: 100 C = {} F.'.format(celsius_to_fahrenheit(100.0)))

In [3]:
# Define a simple quadratic polynomial:
def x_squared(x):
    return x*x

# Print 5 first squares:
p = x_squared
for i in range(5):
    print(p(i+1))

# A simple function of two variables:
def hypotenuse(x, y):
    return math.sqrt(x**2+y**2)

In [4]:
# Example where we include the option of default parameter values:
def sine_wave(x, A=1.0, omega=1.0, phi=0.0):
    return A * math.sin(omega*x+phi)

# Only calling with the first argument. Sets the others two their default values:
print('sin(pi/2)=', sine_wave(math.pi * 0.5))
# Calling the function with omega set to a different value:
print('sin(pi)=', sine_wave(math.pi * 0.5, omega=2.0))
# Should also produce sin(pi)=0:
print('sin(pi)=', sine_wave(math.pi * 0.5, 1.0, 2.0, 0.0))

# We can require the caller of a function to use keywords when setting
# optional input parameters (rather than just ensuring the right order):
def cosine_wave(x, *, A=1.0, omega=1.0, phi=0.0):
    return A*math.cos(omega*x+phi)

print('cos(pi/2)=', cosine_wave(math.pi, omega=0.5))

# Uncommenting the next line produces an error:
#cosine_wave(math.pi, 1.0, 0.5, 0.0)

In [5]:
# Note that since we used the math library version of sine and cosine,
# the above functions are not vectorized:
x = np.linspace(0, 6, 10)
#print(cosine_wave(x))  # <-- produces an error

# Better to do it this way:
def cosine_wave2(x, *, A=1.0, omega=1.0, phi=0.0):
    return A*np.cos(omega*x+phi)
print(cosine_wave2(x))

In the next example, we have added detailed documentation for how to use a
custom function:

In [6]:
def vertical_position_gravity(t, *, h0=10.0):
    """
    Compute vertical position at time t of object dropped from a height h > 0 at time t=0.
    Assume no other forces than gravity, with a constant gravitational acceleration,
    g = 9.81 m/s^2.

    :param t: Time(s) in seconds. Either a single number, or an array.
    :param h0: Height above ground in meters at time t=0 (default: 10.0 m).
    :return: Height above ground in meters at t >= 0.
    """
    h_above = h0-0.5*9.81*t**2
    h_below = np.zeros_like(h_above)
    return np.where(h_above < 0.0, h_below, h_above)

If we need to read the documentation somewhere else, we can type:

In [7]:
help(vertical_position_gravity)

Also observe how the usage of the NumPy special functions *zeros\_like* and
*where* allows us to take both single numbers and arrays as input:

In [8]:
# How long does it take for a ball to drop to the ground?
h_1sec = vertical_position_gravity(1.0, h0=100.0)

t = np.linspace(0, 10.0, 100)
h_t = vertical_position_gravity(t, h0=100.0)
fig_drop_ball = plt.figure()
plt.xlabel('Time (seconds)')
plt.ylabel('Height above ground (m)')
plt.plot(t, h_t)

Functions can take any object as input, and return any kind of object as
output, including other functions:

In [9]:
def NumericalDerivative(f, dx):
    # Notice the usage of an 'anonymous' (lambda) function:
    return lambda x: (f(x + dx) - f(x)) / dx

g = NumericalDerivative(np.sin, 1.0e-3)  # should be ~cos(x)

x_values = [np.pi * 0.25 * i for i in range(5)]  # 0, pi/4, pi/2, ..., 2*pi
for x in x_values:
    diff = g(x) - np.cos(x)
    print('Error in approximation for cos({}) is {}.'.format(x, diff))

## Lambda functions

Notice also in the last example that we defined a lambda function inside
of another function. Lambda functions are 'anonymous functions' that have
a more concise syntax than ordinary Python functions (which are declared by
the 'def' keyword). They are also more restrictive than regular functions,
in that they are only allowed to contain a single expression. The required
syntax is as follows:

        lambda argument(s): expression


The function itself has no name, but we can refer to it via a variable as
in the following example:

In [10]:
func_object = lambda x: x**3

Subsequently, we can call it like any other function:

In [11]:
print(func_object(3.0))

Note that the number of arguments is not restricted to one, e.g.:

In [12]:
func_obj = lambda x,y : x**2+y**2
print(func_obj(1,3))

Why would we want to use such a function in the first place? One typical
situation is the one encountered above, i.e., when we want to return a
function from another function. In that situation we could also have
defined an ordinary function inside the local scope of the outer function:

In [13]:
def numerical_derivative(f, h):
    def inner_function(x):
        return (f(x+h)-f(x))/h
    return inner_function

g = numerical_derivative(np.sin, 1.0e-5)

However, the lambda expression is clearly more concise. Lambda functions
can also be used *inline*:

In [14]:
print('Sum =', (lambda x,y : x+y)(2,3))

## Local versus global

At this point it can be worthwhile to consider how Python functions treat
local versus global variables. For example, take a look at the following
code:

In [15]:
x = 4
def increment(x):
    x += 1  # x is a local variable within the function body
    return x

increment(x)
print(x)

Note that the variable x set before calling the function is *not* altered
within the function body. That is because Python assumes that a variable
*assigned* within a function is a local variable. On the other hand, if we
had only *referenced* x it is implicitly assumed that $x$ is a global variable:

In [16]:
x = 4
def print_global():
    print(x)  # Now x refers to a global varable
print_global()

To see this more clearly, let us define:

In [17]:
x = 4
def print_and_increment_global():
    print(x)
    x = x+1
## Uncomment this line. What happens, and why?
##print_and_increment_global()

You can read more these things
[here](https://docs.python.org/3/faq/programming.html#what-are-the-rules-for-local-and-global-variables-in-python).

## Be careful with mutable function arguments

Suppose we wish to make a function that takes a Python list as input, and
appends the number 9 to the end of the list, before returning it. Assume
moreover that the input list is empty by default,i .e., if no argument is
provided to the function. A naive implementation would be:

In [18]:
def add_nine(input_list=[]):
    input_list.append(9)
    return input_list

However, if we place several successive calls to this function we get the
following output:

In [19]:
print(add_nine())
print(add_nine())
print(add_nine())

What is going on? The reason is that default parameter values used in Python
functions are actually only evaluated once, typically at the time when the
[module](https://www.tutorialspoint.com/python/python_modules.htm)
containing the function is loaded. Thus, after calling the function
the first time, Python takes the object already initialized to an empty list
and adds an element to it, effectively changing the default parameter value
for subsequent function calls. This is not a problem for immutable object
types such as numbers and strings, but for lists, dictionaries etc.
is typically causes unwanted behaviour.

The recommended way of defining the function is:

In [20]:
def add_nine(input_list=None):
    list = [] if input_list is None else input_list
    list.append(9)
    return list

Now we get the behaviour that most people would expect:

In [21]:
print(add_nine())
print(add_nine())
print(add_nine())